In [18]:
import sys
#!{sys.executable} -m pip install pandas
#!{sys.executable} -m pip install numpy
#!{sys.executable} -m pip install sklearn
#!{sys.executable} -m pip install seaborn
#!{sys.executable} -m pip install matplotlib

In [19]:
import pandas as pd
import numpy as np
import sklearn
import re
import string
import seaborn as sn
import matplotlib.pyplot as plt

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [21]:
data = pd.read_csv('./data/train.csv')
data = data.drop(columns=['id', 'keyword', 'location'])

print(data.head(5))

                                                text  target
0  Our Deeds are the Reason of this #earthquake M...       1
1             Forest fire near La Ronge Sask. Canada       1
2  All residents asked to 'shelter in place' are ...       1
3  13,000 people receive #wildfires evacuation or...       1
4  Just got sent this photo from Ruby #Alaska as ...       1


In [22]:
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

def remove_html(text):
    html = re.compile(r'<.*?>')
    return html.sub(r'',text)

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_at(text):
    at = re.compile(r'\@(\w+)')
    return at.sub(r'', text)

def remove_punct(text):
    table = str.maketrans('','',string.punctuation)
    return text.translate(table)

In [23]:
def cleanTweets(data):
    text = data['text']
    text = text.apply(lambda x: remove_URL(x)) \
               .apply(lambda x: remove_html(x)) \
               .apply(lambda x: remove_emoji(x)) \
               .apply(lambda x: remove_at(x)) \
               .apply(lambda x: remove_punct(x))
    return text.to_numpy()

def obtainTargets(data):
    return data['target'].to_numpy()

In [24]:
X_data = cleanTweets(data)
y_data = obtainTargets(data)

print(X_data)
print(y_data)

['Our Deeds are the Reason of this earthquake May ALLAH Forgive us all'
 'Forest fire near La Ronge Sask Canada'
 'All residents asked to shelter in place are being notified by officers No other evacuation or shelter in place orders are expected'
 ... 'M194 0104 UTC5km S of Volcano Hawaii '
 'Police investigating after an ebike collided with a car in Little Portugal Ebike rider suffered serious nonlife threatening injuries'
 'The Latest More Homes Razed by Northern California Wildfire  ABC News ']
[1 1 1 ... 1 1 1]


In [25]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2)

In [26]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X_train)
y = y_train

In [32]:
#print(vectorizer.get_feature_names_out())
print(X.shape)
print(type(X[0][0]))

(6090, 13708)
<class 'scipy.sparse.csr.csr_matrix'>


In [ ]:
logistic_regression_model = LogisticRegression(verbose=3, solver="saga").fit(X, y)

In [ ]:
print(logistic_regression_model.score(vectorizer.transform(X_test), y_test))

In [ ]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(logistic_regression_model.predict(vectorizer.transform(X_test)), y_test).ravel()
cm = np.array([[tn, fp], [fn, tp]])
print(cm)

In [ ]:
df_cm = pd.DataFrame(cm, ['Not a disaster', 'Disaster'], ['Not a disaster', 'Disaster'])
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm, annot=True, square="True", cmap="Blues", fmt="d")
plt.title("Logistic regression confusion matrix\n", fontsize=18)
plt.show()